In [1]:
import os
import numpy as np
import cv2
import json
from src.visualizer import CameraVisualizer
from src.utils import elu_to_c2w, cartesian_to_spherical, spherical_to_cartesian

def load_image(fpath):
    img = cv2.imread(fpath)
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


def add_sph(sph0, sph1):
    return [sph0[0] + sph1[0], (sph0[1] + sph1[1]) % (np.pi *2), sph0[2] + sph1[2]]


colors = []
legends = []
poses = []
images = []

data_root = 'data/omni3d/dinosaur_002'
pose_root = 'outputs/omni3d/dinosaur_002_011+001+077+065'

# data_root = 'data/omni3d/ornaments_001'
# pose_root = 'outputs/omni3d/ornaments_001_063+010+082+001'

# data_root = 'data/omni3d/house_001'
# pose_root = 'outputs/omni3d/house_001_094+015+025+099'

# data_root = 'data/real/toyduck_photo'
# pose_root = 'outputs/real/toyduck_photo_002+004'

# data_root = 'data/real/chair_photo'
# pose_root = 'outputs/real/chair_photo_004+000'

# data_root = 'data/real/foosball_photo'
# pose_root = 'outputs/real/foosball_photo_002+003'

''' A default anchor spherical point (polar, azimuth, radius), used if the ground truth pose is not available. '''
anchor_sph = np.array([1.57, 0, 5])

with open(os.path.join(pose_root, 'pose.json'), 'r') as fin:
    jdata = json.load(fin)

for vid in jdata['obs']:

    img_path = os.path.join(data_root, 'images', f'{vid}.png')
    img = load_image(img_path) if os.path.exists(img_path) else None
    
    gt_pose_path = os.path.join(data_root, 'poses', f'{vid}.npy')
    gt_c2w = np.load(gt_pose_path) if os.path.exists(gt_pose_path) else None
    
    if vid == jdata['anchor_vid']:
        
        if 'elev' in jdata['obs'][vid]:
            anchor_sph[0] = jdata['obs'][vid]['elev']
        
        if gt_c2w is None:
            anchor_eye = spherical_to_cartesian(anchor_sph)
            gt_c2w = elu_to_c2w(anchor_eye, np.zeros(3), np.array([0., 0., 1.]))
        
        else:
            eye = gt_c2w[:3, -1].copy()
            anchor_sph = cartesian_to_spherical(eye)

        poses.append(gt_c2w)
        colors.append('red')
        legends.append('Anchor ' + str(vid))
        images.append(img)
        
        continue

    pred_rel_sph = jdata['obs'][vid]['rel_sph']
    pred_sph = add_sph(pred_rel_sph, anchor_sph)

    pred_xyz = spherical_to_cartesian(pred_sph)
    pred_c2w = elu_to_c2w(pred_xyz, np.zeros(3), np.array([0., 0., 1.]))

    poses.append(pred_c2w)
    colors.append('green')
    legends.append('Est ' + str(vid))
    images.append(img)
    
    if gt_c2w is not None:
        poses.append(gt_c2w)
        colors.append('blue')
        legends.append('GT ' + str(vid))
        images.append(None)


scene_size = anchor_sph[2] * 2.0

vizer = CameraVisualizer(poses, legends, colors, images=images)
fig = vizer.update_figure(scene_bounds=scene_size, base_radius=1.0, show_background=True, show_grid=True, show_ticklabels=True)

fig.show()